In [2]:
import random
import sqlite3

def generate_and_save_mTSP_instances_to_db(nr_instances, nr_cities, nr_salesmen = 1, coord_range=(0, 100), db_file="mTSP_instances.sqlite3"):

    # Connect to the database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create the tables
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS instances (
            instance_id INTEGER PRIMARY KEY,
            nr_cities INTEGER,
            nr_salesmen INTEGER
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cities (
            city_id INTEGER,
            instance_id INTEGER,
            x INTEGER,
            y INTEGER,
            PRIMARY KEY (city_id, instance_id),
            FOREIGN KEY (instance_id) REFERENCES instances(instance_id)
        )
    """)

    # Find next instance_id
    cursor.execute("SELECT MAX(instance_id) FROM instances")
    result = cursor.fetchone()
    next_instance_id = result[0] + 1 if result[0] is not None else 1

    # Create and save instances
    for instance_id in range(next_instance_id, next_instance_id + nr_instances):

        # instace
        cursor.execute("INSERT INTO instances (instance_id, nr_cities, nr_salesmen) VALUES (?, ?, ?)",
                       (instance_id, nr_cities, nr_salesmen))

        # cities
        for city_id in range(nr_cities + 1):
            x = random.randint(coord_range[0], coord_range[1])
            y = random.randint(coord_range[0], coord_range[1])
            cursor.execute("INSERT INTO cities (city_id, instance_id, x, y) VALUES (?, ?, ?, ?)",
                           (city_id, instance_id, x, y))

    conn.commit()
    conn.close()

    print(f"{nr_instances} mTSP instances were created and added to {db_file}.")


generate_and_save_mTSP_instances_to_db(nr_instances=1, nr_cities=17, nr_salesmen=5, coord_range=(0, 100), db_file="train_mTSP.sqlite3")

1 mTSP instances were created and added to train_mTSP.sqlite3.


In [3]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import sqlite3

def solve_tsp_with_or_tools(distance_matrix, nr_salesmen):
    num_cities = len(distance_matrix) - 1  # Exclude depot (node 0)

    # If more salesmen than cities, it's infeasible
    if nr_salesmen > num_cities:
        print(f"Infeasible: {nr_salesmen} salesmen for {num_cities} cities.")
        return None, None

    # Create the routing manager and model
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), nr_salesmen, 0)  # Node 0 is the depot
    routing = pywrapcp.RoutingModel(manager)

    # Define cost of each arc
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add a dimension to track the number of nodes visited by each salesman
    def demand_callback(from_index):
        # Demand of 1 except for the depot (node 0)
        from_node = manager.IndexToNode(from_index)
        return 1 if from_node != 0 else 0

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,  # Transit callback index
        0,                      # Slack (no slack)
        [num_cities] * nr_salesmen,  # Maximum nodes each salesman can visit
        True,                   # Start cumul to zero
        "NodeCount"
    )

    node_count_dimension = routing.GetDimensionOrDie("NodeCount")

    # Enforce that each salesman visits at least one city
    for vehicle_id in range(nr_salesmen):
        node_count_dimension.CumulVar(routing.End(vehicle_id)).SetMin(1)

    # Set search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC) 
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)  
    search_parameters.time_limit.seconds = 10

    solution = routing.SolveWithParameters(search_parameters)

    # Extract the solution
    if solution:
        routes = []
        for vehicle_id in range(nr_salesmen):
            route = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))  # Return to depot
            routes.append(route)
        print("Routes:", routes)
        print("Objective value:", solution.ObjectiveValue())
        return routes, solution.ObjectiveValue()
    else:
        print("No solution found.")
        return None, None

def set_optimal_cost_of_instances(db_file="mTSP.sqlite3"):

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Add the optimal cost column to the instances table
    cursor.execute("PRAGMA table_info(instances)")
    columns = [column[1] for column in cursor.fetchall()]
    if "optimal_cost" not in columns:
        cursor.execute("ALTER TABLE instances ADD COLUMN optimal_cost REAL")

    # Fetch all instances
    cursor.execute("SELECT instance_id, nr_salesmen FROM instances WHERE optimal_cost IS NULL")
    instances = cursor.fetchall()

    for (instance_id, nr_salesmen) in instances:
    
        cursor.execute("SELECT x, y FROM cities WHERE instance_id = ?", (instance_id,))
        cities = cursor.fetchall()

        # Create distance matrix
        distance_matrix = [[
            ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for x2, y2 in cities] for x1, y1 in cities]

        # Solve with ortools automatic algorithm
        _, optimal_cost = solve_tsp_with_or_tools(distance_matrix, nr_salesmen)

        cursor.execute(
            "UPDATE instances SET optimal_cost = ? WHERE instance_id = ?",
            (optimal_cost, instance_id)
        )

    conn.commit()
    conn.close()

    print("Evaluation completed. Optimal costs saved to the database.")

set_optimal_cost_of_instances(db_file="train_mTSP.sqlite3")

No solution found.
Routes: [[0, 17, 0], [0, 10, 0], [0, 8, 2, 11, 13, 16, 5, 3, 6, 4, 15, 12, 9, 0], [0, 14, 7, 0], [0, 1, 0]]
Objective value: 533
Evaluation completed. Optimal costs saved to the database.
